In [ ]:
# export
from local.imports import *
from local.core import compose
from local.notebook.export import *
from local.notebook.showdoc import *
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor, Preprocessor
from nbconvert import HTMLExporter
from nbformat.sign import NotebookNotary
from traitlets.config import Config

In [ ]:
# default_exp notebook.export2html
# default_cls_lvl 3

# Converting notebooks to html

> The functions that transform the dev notebooks in the documentation of the library

## Preprocessing notebook

### Cell processors

In [ ]:
#export
def remove_widget_state(cell):
    "Remove widgets in the output of `cells`"
    if cell['cell_type'] == 'code' and 'outputs' in cell:
        cell['outputs'] = [l for l in cell['outputs'] 
                           if not ('data' in l and 'application/vnd.jupyter.widget-view+json' in l.data)]
    return cell

In [ ]:
#export
def hide_cells(cell):
    "Hide `cell` that need to be hidden"
    if check_re(cell, r's*show_doc\(|^\s*#\s*(export)\s+'): 
        cell['metadata'] = {'hide_input': True}
    return cell

In [ ]:
for source in ['show_doc(read_nb)', '# export\nfrom local.core import *']:
    cell = {'cell_type': 'code', 'source': 'show_doc(read_nb)'}
    cell1 = hide_cells(cell.copy())
    assert 'metadata' in cell1
    assert 'hide_input' in cell1['metadata']
    assert cell1['metadata']['hide_input']

cell = {'cell_type': 'code', 'source': '# exports\nfrom local.core import *'}
assert hide_cells(cell.copy()) == cell

In [ ]:
#export
def clean_exports(cell):
    "Remove exports flag from `cell`"
    cell['source'] = re.sub(r'^#\s*exports[^\n]*\n', '', cell['source'])
    return cell

In [ ]:
cell = {'cell_type': 'code', 'source': '# exports\nfrom local.core import *'}
assert clean_exports(cell.copy()) == {'cell_type': 'code', 'source': 'from local.core import *'}
cell = {'cell_type': 'code', 'source': '# exports core\nfrom local.core import *'}
assert clean_exports(cell.copy()) == {'cell_type': 'code', 'source': 'from local.core import *'}

In [ ]:
#export
def treat_backticks(cell):
    "Add links to backticks words in `cell`"
    if cell['cell_type'] == 'markdown': cell['source'] = add_doc_links(cell['source'])
    return cell

In [ ]:
cell = {'cell_type': 'markdown', 'source': 'This is a `Tensor`'}
assert treat_backticks(cell) == {'cell_type': 'markdown',
    'source': 'This is a [`Tensor`](https://pytorch.org/docs/stable/tensors.html#torch-tensor)'}

In [ ]:
# export
def convert_links(cell):
    "Convert the .ipynb links to .html"
    if cell['cell_type'] == 'markdown':
        cell['source'] = re.sub(r'\[([^http][^\]]*)\]\(([^\)]*).ipynb\)', r'[\1](\2.html)', cell['source'])
    return cell

In [ ]:
cell = {'cell_type': 'markdown', 'source': "This is a link to a [notebook](01_core.ipynb)."}
assert convert_links(cell) == {'cell_type': 'markdown', 
                               'source': "This is a link to a [notebook](01_core.html)."}
cell = {'cell_type': 'markdown', 'source': "This is a link to a [page](01_core.html)."}
assert convert_links(cell) == cell
cell = {'cell_type': 'markdown', 'source': "This is a link to an [external nb](http://01_core.ipynb)."}
assert convert_links(cell) == cell

In [ ]:
#export
def add_jekyll_notes(cell):
    "Convert block quotes to jekyll notes in `cell`"
    t2style = {'Note': 'info', 'Warning': 'danger', 'Important': 'warning'}
    def _inner(m):
        title,text = m.groups()
        style = t2style.get(title, None)
        if style is None: return f"> {m.groups()[0]}: m.groups()[1]"
        res = f'<div markdown="span" class="alert alert-{style}" role="alert">'
        return res + f'<i class="fa fa-{style}-circle"></i> <b>{title}: </b>{text}</div>'
    if cell['cell_type'] == 'markdown':
        cell['source'] = re.sub(r'>\s*([^:]*):\s*([^\n]*)(?:\n|$)', _inner, cell['source'])
    return cell

Supported styles are `Warning`, `Note` and `Important`:

> Warning: There will be no second warning!

> Important: Pay attention! This is important.

> Note: Take note of this.

In [ ]:
for w,s in zip(['Warning', 'Note', 'Important', 'Bla'], ['danger', 'info', 'warning', 'info']):
    cell = {'cell_type': 'markdown', 'source': f"> {w}: This is my final {w.lower()}!"}
    res = f'<div markdown="span" class="alert alert-{s}" role="alert">'
    res += f'<i class="fa fa-{s}-circle"></i> <b>{w}: </b>This is my final {w.lower()}!</div>'
    if w != 'Bla': assert add_jekyll_notes(cell) == {'cell_type': 'markdown', 'source': res}
    else: assert add_jekyll_notes(cell) == cell

In [ ]:
text = "![pixelshuffle](images/pixelshuffle.png)"

In [ ]:
re.search('^!\[[^\]]*\]\(([^\)]*)\)', text).groups()

('images/pixelshuffle.png',)

In [ ]:
pat = re.compile(r'^!\[([^\]]*)\]\(([^\)]*)\)', re.MULTILINE)

In [ ]:
#export
def copy_images(cell, fname, dest):
    pat = re.compile(r'^!\[[^\]]*\]\(([^\)]*)\)|<img src="([^"]*)"', re.MULTILINE)
    if cell['cell_type'] == 'markdown' and re.search(pat, cell['source']):
        grps = re.search(pat, cell['source']).groups()
        src = grps[0] or grps[1]
        os.makedirs((Path(dest)/src).parent, exist_ok=True)
        shutil.copy(Path(fname).parent/src, Path(dest)/src)
    return cell

In [ ]:
dest_img = Path('docs')/'images'/'pixelshuffle.png' 
dest_bak = Path('docs')/'images'/'pixelshuffle.bak'
if dest_img.exists(): shutil.move(dest_img, dest_bak)
for text in ['Text\n![Alt](images/pixelshuffle.png)', 
             'Text\n<img src="images/pixelshuffle.png" alt="Pixelshuffle" style="width: 100%; height: auto;"/>']:
    cell = {'cell_type': 'markdown', 'source': text}
    cell = copy_images(cell, Path('10_layers.ipynb'), Path('docs'))
    #Function doesn't touch cell
    assert cell == cell
    #Image has been copied
    assert dest_img.exists()
    os.remove(dest_img)
if dest_bak.exists(): shutil.move(dest_bak, dest_img)

In [ ]:
#export
def remove_hidden(cells):
    "Remove in `cells` the ones with a flag `#hide` or `#default_exp`"
    res = []
    pat = re.compile(r'^\s*#\s*(hide|default_exp)\s+')
    for cell in cells:
        if cell['cell_type']=='markdown' or re.search(pat, cell['source']) is None:
            res.append(cell)
    return res

In [ ]:
cells = [{'cell_type': 'code', 'source': source} for source in [
    '# export\nfrom local.core import *', 
    '# hide\nfrom local.core import *',
    '#exports\nsuper code',
    '#default_exp notebook.export',
    'show_doc(read_nb)']] + [{'cell_type': 'markdown', 'source': 'nice'}]
         
cells1 = remove_hidden(cells)
assert len(cells1) == 4
assert cells1[0] == cells[0]
assert cells1[1] == cells[2]
assert cells1[2] == cells[4]
assert cells1[3] == cells[5]

In [ ]:
# export
def find_default_level(cells):
    "Find in `cells` the default export module."
    for cell in cells:
        tst = check_re(cell, r'^\s*#\s*default_cls_lvl\s*(\d*)\s*$')
        if tst: return int(tst.groups()[0])
    return 2

In [ ]:
tst_nb = read_nb('91_notebook_export.ipynb')
assert find_default_level(tst_nb['cells']) == 3

In [ ]:
re.search(r'show_doc\s*\(\s*([^,\)\s]*)[,\)\s]', 'show_doc(Pipeline.composed), title_level=18)').groups()[0]

'Pipeline.composed'

In [ ]:
#export
def _show_doc_cell(name, cls_lvl=None):
    return {'cell_type': 'code',
            'execution_count': None,
            'metadata': {},
            'outputs': [],
            'source': f"show_doc({name}{'' if cls_lvl is None else f', default_cls_level={cls_lvl}'})"}

def add_show_docs(cells, cls_lvl=None):
    "Add `show_doc` for each exported function or class"
    sd_pat = re.compile(r'^show_doc\s*\(\s*([^,\)\s]*)[,\)\s]', re.MULTILINE)
    documented = [re.search(sd_pat, cell['source']).groups()[0] for cell in cells 
                  if cell['cell_type']=='code' and re.search(sd_pat, cell['source']) is not None]
    res = []
    for cell in cells:
        res.append(cell)
        if check_re(cell, r'^\s*#\s*exports?\s*'):
            names = export_names(cell['source'], func_only=True)
            for n in names: 
                if n not in documented: res.append(_show_doc_cell(n, cls_lvl=cls_lvl))
    return res

In [ ]:
for i,cell in enumerate(tst_nb['cells']):
    if cell['source'].startswith('#export\ndef read_nb'): break
tst_cells = [c.copy() for c in tst_nb['cells'][i-1:i+1]]
added_cells = add_show_docs(tst_cells, cls_lvl=3)
assert len(added_cells) == 3
assert added_cells[0] == tst_nb['cells'][i-1]
assert added_cells[1] == tst_nb['cells'][i]
assert added_cells[2] == _show_doc_cell('read_nb', cls_lvl=3)
assert added_cells[2]['source'] == 'show_doc(read_nb, default_cls_level=3)'

#Check show_doc isn't added if it was already there.
tst_cells1 = [{'cell_type':'code', 'source': '#export\ndef my_func(x):\n    return x'},
              {'cell_type':'code', 'source': 'show_doc(my_func)'}]
assert add_show_docs(tst_cells1) == tst_cells1
tst_cells1 = [{'cell_type':'code', 'source': '#export\ndef my_func(x):\n    return x'},
              {'cell_type':'markdown', 'source': 'Some text'},
              {'cell_type':'code', 'source': 'show_doc(my_func, title_level=3)'}]
assert add_show_docs(tst_cells1) == tst_cells1

In [ ]:
# export
def remove_fake_headers(cells):
    "Remove in `cells` the fake header"
    res = []
    pat = re.compile(r'#+.*-$')
    for cell in cells:
        if cell['cell_type']=='code' or re.search(pat, cell['source']) is None:
            res.append(cell)
    return res

In [ ]:
cells = [{'cell_type': 'markdown',
          'metadata': {},
          'source': '### Fake-'}] + tst_nb['cells'][:10]
cells1 = remove_fake_headers(cells)
assert len(cells1) == len(cells)-1
assert cells1[0] == cells[1]

In [ ]:
# export
def remove_empty(cells):
    "Remove in `cells` the empty cells"
    return [c for c in cells if len(c['source']) >0]

### Grabbing metada

In [ ]:
# export
def get_metadata(cells):
    "Find the cell with title and summary in `cells`."
    pat = re.compile('^\s*#\s*([^\n]*)\n*>\s*([^\n]*)')
    for i,cell in enumerate(cells):
        if cell['cell_type'] == 'markdown':
            match = re.match(pat, cell['source'])
            if match: 
                cells.pop(i)
                return {'keywords': 'fastai',
                        'summary' : match.groups()[1],
                        'title'   : match.groups()[0]}
    return {'keywords': 'fastai',
            'summary' : 'summary',
            'title'   : 'Title'}

In [ ]:
tst_nb = read_nb('91_notebook_export.ipynb')
assert get_metadata(tst_nb['cells']) == {
    'keywords': 'fastai',
    'summary': 'The functions that transform the dev notebooks in the fastai library',
    'title': 'Converting notebooks to modules'}
#The cell with the metada is poped out, so if we do it a second time we get the default.
assert get_metadata(tst_nb['cells']) == {'keywords': 'fastai',
            'summary' : 'summary',
            'title'   : 'Title'}

## Executing show_doc cells

In [ ]:
# export
class ExecuteShowDocPreprocessor(ExecutePreprocessor):
    "An `ExecutePreprocessor` that only executes `show_doc` and `import` cells"
    def preprocess_cell(self, cell, resources, index):
        pat = re.compile(r"^\s*show_doc\(([^\)]*)\)|^\s*#\s*exports?\s*", re.MULTILINE)
        if 'source' in cell and cell['cell_type'] == "code":
            if re.search(pat, cell['source']):
                return super().preprocess_cell(cell, resources, index)
        return cell, resources

In [ ]:
# export
def _import_show_doc_cell(name=None):
    "Add an import show_doc cell + deal with the _file_ hack if necessary."
    source = f"#export\nfrom local.notebook.showdoc import show_doc"
    if name: source += f"\nfrom pathlib import Path\n_file_ = {name}"
    return {'cell_type': 'code',
            'execution_count': None,
            'metadata': {'hide_input': True},
            'outputs': [],
            'source': source}

def execute_nb(nb, metadata=None, show_doc_only=True, name=None):
    "Execute `nb` (or only the `show_doc` cells) with `metadata`"
    nb['cells'].insert(0, _import_show_doc_cell(name))
    ep_cls = ExecuteShowDocPreprocessor if show_doc_only else ExecutePreprocessor
    ep = ep_cls(timeout=600, kernel_name='python3')
    metadata = metadata or {}
    pnb = nbformat.from_dict(nb)
    ep.preprocess(pnb, metadata)
    return pnb

In [ ]:
fake_nb = {k:v for k,v in tst_nb.items() if k != 'cells'}
fake_nb['cells'] = [tst_nb['cells'][0].copy()] + added_cells
fake_nb = execute_nb(fake_nb)
assert len(fake_nb['cells'][-1]['outputs']) > 0

## Conversion

In [ ]:
#hide
#Tricking jupyter notebook to have a __file__ attribute. All _file_ will be replaced by __file__
_file_ = Path('local').absolute()/'notebook'/'export.py'

In [ ]:
# export
def _exporter():
    exporter = HTMLExporter(Config())
    exporter.exclude_input_prompt=True
    exporter.exclude_output_prompt=True
    exporter.template_file = 'jekyll.tpl'
    exporter.template_path.append(str(Path(_file_).parent))
    return exporter

In [ ]:
# export
process_cells = [remove_fake_headers, remove_hidden, remove_empty]
process_cell  = [hide_cells, remove_widget_state, treat_backticks, add_jekyll_notes, convert_links]

In [ ]:
# export
def _find_file(cells):
    "Find in `cells` if a _file_ is defined."
    pat = re.compile(r'^_file_\s*=\s*(\S*)\s*', re.MULTILINE)
    for cell in cells:
        if cell['cell_type']=='code' and re.search(pat, cell['source']): 
            return re.search(pat, cell['source']).groups()[0]

In [ ]:
#hide
tst_nb = read_nb('91_notebook_export.ipynb')
assert _find_file(tst_nb['cells']) == "Path('local').absolute()/'notebook'/'export.py'"

In [ ]:
# export
def convert_nb(fname, dest_path='docs'):
    "Convert a notebook `fname` to html file in `dest_path`."
    fname = Path(fname).absolute()
    nb = read_nb(fname)
    cls_lvl = find_default_level(nb['cells'])
    _name = _find_file(nb['cells'])
    nb['cells'] = compose(*process_cells,partial(add_show_docs, cls_lvl=cls_lvl))(nb['cells'])
    nb['cells'] = [compose(partial(copy_images, fname=fname, dest=dest_path), *process_cell)(c) 
                    for c in nb['cells']]
    fname = Path(fname).absolute()
    dest_name = '.'.join(fname.with_suffix('.html').name.split('_')[1:])
    meta_jekyll = get_metadata(nb['cells'])
    meta_jekyll['nb_path'] = f'{fname.parent.name}/{fname.name}'
    nb = execute_nb(nb, name=_name)
    nb['cells'] = [clean_exports(c) for c in nb['cells']]
    with open(f'{dest_path}/{dest_name}','w') as f:
        f.write(_exporter().from_notebook_node(nb, resources=meta_jekyll)[0])

In [ ]:
# export
def convert_all(path='.', dest_path='docs', force_all=False):
    "Convert all notebooks in `path` to html files in `dest_path`."
    path = Path(path)
    changed_cnt = 0
    for fname in path.glob("*.ipynb"):
        # only rebuild modified files
        if fname.name.startswith('_'): continue
        fname_out = Path(dest_path)/'.'.join(fname.with_suffix('.html').name.split('_')[1:])
        if not force_all and fname_out.exists() and os.path.getmtime(fname) < os.path.getmtime(fname_out): 
            continue
        print(f"converting: {fname} => {fname_out}")
        changed_cnt += 1
        try: convert_nb(fname, dest_path=dest_path)
        except: print("Failed")
    if changed_cnt==0: print("No notebooks were modified")

In [ ]:
#hide
convert_all()

converting: 04_data_core.ipynb => docs/data.core.html
converting: 01_core.ipynb => docs/core.html
converting: 08_augmentation.ipynb => docs/augmentation.html
converting: 92_notebook_showdoc.ipynb => docs/notebook.showdoc.html
Failed
converting: 12_learner.ipynb => docs/learner.html
converting: 11_optimizer.ipynb => docs/optimizer.html
converting: 05_data_source.ipynb => docs/data.source.html
converting: 10_layers.ipynb => docs/layers.html
converting: 06_vision_core.ipynb => docs/vision.core.html
converting: 02_data_pipeline.ipynb => docs/data.pipeline.html
converting: 07_pets_tutorial.ipynb => docs/pets.tutorial.html
converting: 93_notebook_export2html.ipynb => docs/notebook.export2html.html


In [ ]:
convert_nb('12_learner.ipynb')

## Export-

In [ ]:
#hide
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 02_data_pipeline.ipynb.
Converted 03_data_external.ipynb.
Converted 04_data_core.ipynb.
Converted 05_data_source.ipynb.
Converted 06_vision_core.ipynb.
Converted 07_pets_tutorial.ipynb.
Converted 08_augmentation.ipynb.
Converted 10_layers.ipynb.
Converted 11_optimizer.ipynb.
Converted 12_learner.ipynb.
Converted 90_notebook_core.ipynb.
Converted 91_notebook_export.ipynb.
Converted 92_notebook_showdoc.ipynb.
Converted 93_notebook_export2html.ipynb.
Converted 94_index.ipynb.


In [ ]:
#hide
def debug_nb(fname, dest=None):
    fname = Path(fname).absolute()
    nb = read_nb(fname)
    cls_lvl = find_default_level(nb['cells'])
    _name = _find_file(nb['cells'])
    nb['cells'] = compose(*process_cells, partial(add_show_docs, cls_lvl=cls_lvl))(nb['cells'])
    nb['cells'] = [compose(*process_cell)(c) for c in nb['cells']]
    fname = Path(fname).absolute()
    nb = execute_nb(nb, name=_name)
    dest = dest or fname.with_suffix('.dbg.ipynb')
    nbformat.write(nbformat.from_dict(nb), open(dest, 'w'), version=4)

In [ ]:
#hide
debug_nb('93_notebook_export2html.ipynb')

In [ ]:
if not 1==2: print("True")

True
